In [ ]:
%load_ext autoreload
%autoreload 2

import util.replay_buffer as replay_buffer
import util.rl_path as rl_path
from util.rl_path import compute_g
import pickle
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [ ]:
compute_g

In [ ]:
# import train data and make into new path
train_data = pickle.load(open("paths.5.half.pkl", "rb"))
train_data2 = [rl_path.RLPath2(path, compute_g) for path in tqdm(train_data)]

In [ ]:
buf = replay_buffer.ReplayBuffer(10000000)
for path in tqdm(train_data2):
    buf.store(path)

In [ ]:
# gather init states
init_states = []
eval_paths = []
agent._env.seed(606)
for _ in tqdm(range(1000)):
    path = rl_path.RLPath2(agent._rollout_path(True), compute_g)
    eval_paths.append(path)
    init_states.append(path.states[0])
init_states = np.stack(init_states)

In [ ]:
from util.fqe import FittedQEvaluation as FQE

In [ ]:
import awr_configs
import learning.awr_agent as awr_agent
import gym
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import torch
import util.rl_path as rl_path
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
from util.policy import Policy

In [ ]:
configs = awr_configs.AWR_CONFIGS['LunarLanderContinuous-v2']

In [ ]:
env = gym.make("LunarLanderContinuous-v2")
graph = tf.Graph()
sess = tf.Session(graph=graph)
agent = awr_agent.AWRAgent(env=env, sess=sess, **configs)
agent.load_model("output/model.ckpt")

In [ ]:
# run FQE
fqe = FQE(agent._discount, agent.get_state_size(), agent.get_action_size(), 0.001)
policy = Policy(agent)
# c for cost
# g for constraint
# last argument is index of constraint
fqe.run(policy, "c", buf, init_states, 200, 0)

In [ ]:
# exact policy eval
costs = []
for path in eval_paths:
    path.set_cost(buf.scale, "c", 0)
    costs.append(path.discounted_sum(agent._discount, "costs"))
costs = np.array(costs)
costs.mean()

In [ ]:
# FQE eval
S = init_states
A = policy(S)
SA = np.hstack([S, A]) 
predicts = fqe.Q(SA)
predicts.mean()

In [ ]:
plt.scatter(costs, predicts, color="r")
plt.xlabel("True Value")
plt.ylabel("Prediction")
plt.show()